In [65]:
import pandas as pd
import os
import re
import numpy as np
from os import path
import datetime

In [2]:
data_dir='/Users/coconut/code/DataBase/data/news-data'

In [15]:
os.chdir(data_dir)

获取目标文件
```sh
grep "航空" ./brief/* -r | awk -F.csv '{print$1}' > Aerial/briefhit
```

查看同个文件中出现航空的次数  
```sh
sort target| uniq -c
```

去重导出到文件
```sh
sort Aerial/briefhit| uniq | awk '{print $1" "$2".csv" }' > Aerial/brief_target
sort Aerial/majorhit| uniq | awk '{print $1" "$2".csv" }' > Aerial/major_target
```

过滤掉文件中混杂的不同日期数据

In [161]:
def filter_date(x, date):
    if x.startswith(date):
        return x
    else:
        return np.nan


def find_target(x):
    if pd.notna(x) and "航空" in x:
        return x
    else:
        return np.nan

抽取出目标内容

In [141]:
c = 0
df = pd.DataFrame()
with open(path.join(data_dir,'major_target')) as f:
    for i in f.readlines():
        c+=1
        data = pd.read_csv(i.replace('\n',''))
        data.content = data.content.apply(find_target)
        data.pub_time = data.pub_time.apply(filter_date, date=i.split(" ")[0].split("/")[-1])
        data.dropna(subset=['content', 'pub_time'], inplace=True)
        df = df.append(data[['title','content','pub_time','src']])

        
# 得到的索引是重复的，需要重整index
df.index = [i for i in range(df.shape[0])]

In [142]:
df.to_csv("Aerial_Major.csv", index=False)

In [144]:
pd.read_csv("./brief/10jqka/2020-06-20 00:00:00.csv").columns

Index(['Unnamed: 0', 'datetime', 'content', 'title'], dtype='object')

In [165]:
c = 0
df2 = pd.DataFrame()
with open(path.join(data_dir,'brief_target')) as f:
    for i in f.readlines():
        data = pd.read_csv(i.replace('\n',''))
        data.content = data.content.apply(find_target)
        data.datetime = data.datetime.apply(filter_date, date=i.split(" ")[0].split("/")[-1])
        data.dropna(subset=['content', 'datetime'], inplace=True)
        df2 = df2.append(data[['title', 'content', 'datetime']])

        
# 得到的索引是重复的，需要重整index
df2.index = [i for i in range(df2.shape[0])]

In [166]:
df2

,title,content,datetime
0,美多家航空公司继续推迟复飞波音737MAX波音公司,据外媒报道，鉴于波音公司上个月表示，737MAX系列客机无望在今年年中前获得复飞许可，近日，...,2020-02-17 21:10:00
1,东方航空：计划恢复国际航线5条 国内航线163条,东方航空23日消息，为保障全国企业有序复工及旅客顺利便捷出行，东航计划恢复国际航线5条，国内...,2020-02-23 20:31:00
2,美国联邦航空管理局周二提议 航空公司应完成对波音 737 ...,据报道，美国联邦航空管理局周二提议，在波音 737 MAX客机恢复服务以前，航空公司应完成对...,2020-02-26 03:52:00
3,海航“被接管”？权威知情人士说不能“想当然”,权威知情人士称，海航集团不会被接管，海南航空也不会被拆分。（第一财经）,2020-02-29 20:32:00
4,海南航空不会被海航集团出售,从知情人士处获悉，海南省海航集团联合工作组成立后，海航旗下的航空资产不会以拆分出售资产的形式...,2020-02-29 18:06:00
...,...,...,...
6548,中国造船业已摘取造船业“三颗明珠” 中的两颗,云财经讯，航空母舰、豪华邮轮、大型液化天然气运输船，是全球造船业皇冠上的“三颗明珠”。沪东中...,2020-11-04 13:08:13
6549,全球第一台飞行汽车获批上路,云财经讯，据媒体报道，飞行汽车Liberty近日已获得批准可以在欧洲道路上行驶，这是全球第一...,2020-11-04 07:54:49
6550,道指初步收涨逾500点,云财经讯，道指初步收涨逾500点。高盛和摩根士丹利均涨约4%，领跑大型银行股。主要科技股里，...,2020-11-04 04:57:29
6551,民航局：支持粤港澳大湾区内机场间深化互利共赢合作,云财经讯，11月4日，中国民用航空局局长冯正霖在京会见香港特区行政长官林郑月娥一行，双方就深...,2020-11-05 14:00:30


In [167]:
df2.to_csv("Aerial_Brief.csv")